In [ ]:
# 导入依赖
import matplotlib.pyplot as plt
from mzm.model import simulate_mzm
from mzm.utils import cleanup_torch
from mzm.plot import plot_optical_spectrum_osa, plot_electrical_spectrum, plot_bias_scan

%config InlineBackend.figure_format = 'retina'

In [ ]:
# 1. 运行一次仿真（参数等价于 MATLAB 脚本）
sim = simulate_mzm(
    SymbolRate=10e9,
    Fs=100e9,
    T_total=10e-6,
    Vpi_RF=5.0,
    Vpi_DC=5.0,
    ER_dB=30.0,
    IL_dB=6.0,
    Responsivity=0.786,
    R_load=50.0,
    Pin_dBm=10.0,
    Temp_K=290.0,
    RIN_dB_Hz=-145.0,
    f_rf=1e9,
    V_rf_amp=0.2,
    V_bias=2.5,
    vpi_compatible_dbm=False,
)

# 释放中间缓存（可选）
cleanup_torch()

In [ ]:
# 2. 输出光谱（OSA 视图，自动标注载波与 ±1/±2 阶）
plot_optical_spectrum_osa(sim, f_rf_hz=1e9, max_order=2)
plt.show()

In [ ]:
# 3. 带噪底的电谱（自动标注 DC/1×/2×RF）
plot_electrical_spectrum(sim, f_rf_hz=1e9, harmonic_orders=(0, 1, 2))
plt.show()

In [ ]:
# 4. 偏压扫描曲线
plot_bias_scan(sim)
plt.show()

In [ ]:
# 5. PD 输出 DC/1×RF/2×RF 功率 vs 偏压（dBm）
import numpy as np

# 为了让 sweep 运行更快，这里用更短的仿真窗口与更低采样率。
# 要覆盖到 2×RF，Fs 需要 > 4×f_rf（这里留裕量）。
f_rf = 1e9
Fs_sweep = 20e9
T_total_sweep = 2e-5
n_points = 1601  # 可改大（更平滑）/改小（更快）

# 扫描范围 [-2Vpi, 2Vpi]
Vpi_DC = 5.0
V_scan = np.linspace(-2.0 * Vpi_DC, 2.0 * Vpi_DC, n_points)

p_dc_dBm = np.empty_like(V_scan, dtype=np.float64)
p_1f_dBm = np.empty_like(V_scan, dtype=np.float64)
p_2f_dBm = np.empty_like(V_scan, dtype=np.float64)

for k, Vb in enumerate(V_scan):
    sim_k = simulate_mzm(
        SymbolRate=10e9,
        Fs=Fs_sweep,
        T_total=T_total_sweep,
        Vpi_RF=5.0,
        Vpi_DC=Vpi_DC,
        ER_dB=30.0,
        IL_dB=6.0,
        Responsivity=0.786,
        R_load=50.0,
        Pin_dBm=10.0,
        Temp_K=290.0,
        RIN_dB_Hz=-145.0,
        f_rf=f_rf,
        V_rf_amp=0.05,
        V_bias=float(Vb),
        vpi_compatible_dbm=False,
    )
    f_e = sim_k.spectrum.f_elec
    p_e = sim_k.spectrum.P_elec_spec_dBm

    # DC/1×RF/2×RF 在电谱上的采样点
    idx_dc = int(np.argmin(np.abs(f_e - 0.0)))
    idx_1 = int(np.argmin(np.abs(f_e - f_rf)))
    idx_2 = int(np.argmin(np.abs(f_e - 2.0 * f_rf)))

    p_dc_dBm[k] = float(p_e[idx_dc])
    p_1f_dBm[k] = float(p_e[idx_1])
    p_2f_dBm[k] = float(p_e[idx_2])

def _report_min_max(name: str, y: np.ndarray, x: np.ndarray) -> None:
    i_min = int(np.argmin(y))
    i_max = int(np.argmax(y))
    print(
        f"[{name}] min={y[i_min]:.2f} dBm @ V={x[i_min]:.4f} V | "
        f"max={y[i_max]:.2f} dBm @ V={x[i_max]:.4f} V"
    )

_report_min_max("PD DC", p_dc_dBm, V_scan)
_report_min_max("PD 1×RF", p_1f_dBm, V_scan)
_report_min_max("PD 2×RF", p_2f_dBm, V_scan)

cleanup_torch()

plt.figure(figsize=(10, 6))
plt.plot(V_scan, p_dc_dBm, label='PD DC', linewidth=2)
plt.plot(V_scan, p_1f_dBm, label='PD 1×RF', linewidth=2)
plt.plot(V_scan, p_2f_dBm, label='PD 2×RF', linewidth=2)
plt.title('PD Output Power vs Bias (dBm)')
plt.xlabel('Bias Voltage (V)')
plt.ylabel('Power (dBm)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 6. 仅导频(10MHz)的 PD DC/1f/2f 功率 vs 偏压（dBm），不加射频
import numpy as np
import torch
import matplotlib.pyplot as plt
import mzm.model as model
from mzm.utils import cleanup_torch

# --- Dither (pilot) settings ---
f_dither = 1e7          # 10 MHz 导频
V_dither_amp = 0.05     # 导频幅度 (V_peak)

# --- RF settings (disabled) ---
V_rf_amp = 0.0          # 关 RF（不注入 1GHz 射频）
f_rf = 1e9              # 仅为接口占位，V_rf_amp=0 时无效

# --- Device / measurement parameters ---
Pin_dBm = 10.0          # 进入 MZM 的输入光功率
Responsivity = 0.786    # A/W
R_load = 50.0           # Ohm
pd_tap = 0.10           # 1:9 光耦，10% 功率口接 PD（MZM 输出后、PD 前）

# --- Sampling / integration ---
Fs = 200e6              # 采样率 (Hz)，对 10MHz 足够
n_periods = 100          # 积分导频周期数：T = n_periods / f_dither
T_total = n_periods / f_dither
N = int(round(Fs * T_total))
RBW_est = Fs / max(N, 1)
print(f"[dither] f_dither={f_dither:.0f} Hz, n_periods={n_periods}, Fs={Fs:.0f} Hz, T_total={T_total:.6f} s, N={N}, RBW≈{RBW_est:.2f} Hz")
print(f"[coupler] Pin_dBm(in)={Pin_dBm:.2f} dBm, pd_tap={pd_tap*100:.0f}% (after MZM -> PD)")

# --- Bias sweep ---
Vpi_DC = 5.0
n_points = 1601
V_scan = np.linspace(-2.0 * Vpi_DC, 2.0 * Vpi_DC, n_points)
V_bias_t = torch.tensor(V_scan, dtype=torch.float64, device=torch.device("cpu"))

# 1f/2f 功率（dBm）
p1_dBm_t, p2_dBm_t = model.measure_pd_dither_1f2f_dbm_batch_torch(
    V_bias=V_bias_t,
    V_dither_amp=V_dither_amp,
    f_dither=f_dither,
    Fs=Fs,
    n_periods=n_periods,
    Vpi_DC=Vpi_DC,
    ER_dB=30.0,
    IL_dB=6.0,
    Pin_dBm=Pin_dBm,
    pd_tap=pd_tap,
    Responsivity=Responsivity,
    R_load=R_load,
    V_rf_amp=V_rf_amp,
    f_rf=f_rf,
 )

# DC（用 pd_dc 估计直流耗散功率，再转 dBm）
_, _, pd_dc_t = model.measure_pd_dither_normalized_batch_torch(
    V_bias=V_bias_t,
    V_dither_amp=V_dither_amp,
    f_dither=f_dither,
    Fs=Fs,
    n_periods=n_periods,
    Vpi_DC=Vpi_DC,
    ER_dB=30.0,
    IL_dB=6.0,
    Pin_dBm=Pin_dBm,
    pd_tap=pd_tap,
    Responsivity=Responsivity,
    R_load=R_load,
    V_rf_amp=V_rf_amp,
    f_rf=f_rf,
 )

# --- “进入 PD 的光功率”（已包含 pd_tap 分光） ---
# pd_dc 是平均光电流 (A)，进入 PD 的平均光功率：P_pd_avg = pd_dc / Responsivity (W)
P_pd_in_W_t = pd_dc_t.to(torch.float64) / float(Responsivity)
P_pd_in_dBm_t = 10.0 * torch.log10(P_pd_in_W_t * 1000.0 + 1e-30)

# DC 电功率（电阻耗散）
pdc_W_t = (pd_dc_t.to(torch.float64) ** 2) * float(R_load)
pdc_dBm_t = 10.0 * torch.log10(pdc_W_t * 1000.0 + 1e-30)

# to numpy
P_pd_in_dBm = P_pd_in_dBm_t.detach().cpu().numpy()
pdc_dBm = pdc_dBm_t.detach().cpu().numpy()
p1_dBm = p1_dBm_t.detach().cpu().numpy()
p2_dBm = p2_dBm_t.detach().cpu().numpy()

def _report_min_max(name: str, y: np.ndarray, x: np.ndarray) -> None:
    i_min = int(np.argmin(y))
    i_max = int(np.argmax(y))
    print(
        f"[{name}] min={y[i_min]:.2f} dBm @ V={x[i_min]:.4f} V | "
        f"max={y[i_max]:.2f} dBm @ V={x[i_max]:.4f} V"
    )

# 进入 PD 的平均光功率（dBm）
_report_min_max(f"Optical Power into PD (avg, tap={pd_tap*100:.0f}%)", P_pd_in_dBm, V_scan)

# 电域功率/谐波功率
_report_min_max("PD DC (elec, from Idc)", pdc_dBm, V_scan)
_report_min_max(f"PD 1f @ {f_dither/1e6:.0f}MHz", p1_dBm, V_scan)
_report_min_max(f"PD 2f @ {2*f_dither/1e6:.0f}MHz", p2_dBm, V_scan)

cleanup_torch()

plt.figure(figsize=(10, 6))
plt.plot(V_scan, P_pd_in_dBm, label=f"Optical into PD (avg, tap={pd_tap*100:.0f}%, dBm)", linewidth=2, linestyle="--")
plt.plot(V_scan, pdc_dBm, label="PD DC (elec, dBm)", linewidth=2)
plt.plot(V_scan, p1_dBm, label=f"PD 1f @ {f_dither/1e6:.0f}MHz (dBm)", linewidth=2)
plt.plot(V_scan, p2_dBm, label=f"PD 2f @ {2*f_dither/1e6:.0f}MHz (dBm)", linewidth=2)
plt.title(f"PD Output (Dither-only, no RF): f_dither={f_dither/1e6:.0f} MHz, {V_dither_amp*1e3*2:.0f} mVpp, tap={pd_tap*100:.0f}%")
plt.xlabel("Bias Voltage (V)")
plt.ylabel("Power (dBm)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()